In [10]:
import pyproj 
import math
import requests
import os
from pathlib import Path
import rasterio
import numpy as np

In [10]:
def get_highest_resolution_bbox(bbox_wgs84, dest_crs, pix_size_in_dest_crs):
    split_bbox = bbox_wgs84.split(',')
    min_x = float(split_bbox[0])
    min_y = float(split_bbox[1])
    max_x = float(split_bbox[2])
    max_y = float(split_bbox[3])
    
    # convert bbox to web mercator
    project = pyproj.Transformer.from_crs("EPSG:4326", dest_crs, always_xy=True)
    min_x, min_y = project.transform(min_x, min_y, direction='forward')
    max_x, max_y = project.transform(max_x, max_y, direction='forward')
    
    width = max_x - min_x
    height = max_y - min_y
    width = width / pix_size_in_dest_crs
    height = height / pix_size_in_dest_crs
    
    return (f"{min_x},{min_y},{max_x},{max_y}", (width, height))

In [12]:
bbox, resolution = get_highest_resolution_bbox("-124.947510,41.841704,-116.378174,46.361555", "EPSG:3857", 30)

print(bbox, resolution)

-13909093.189087456,5137296.885289235,-12955159.069130989,5838479.182471741 (31797.80399854891, 23372.743239416865)


Feed the bbox, 

In [40]:
width = 5000
height = 5000

starting_x = -13909093
starting_y = 5137296

ending_x = -12955159
ending_y = 5838479

pix_size = 30

diff_x = ending_x - starting_x
diff_y = ending_y - starting_y

num_tiles_x = math.ceil(diff_x / width / pix_size)
num_tiles_y = math.ceil(diff_y / height / pix_size)   

print(f"num_tiles_x: {num_tiles_x}, num_tiles_y: {num_tiles_y}, total {num_tiles_x * num_tiles_y} tiles")

bbox_list = []

for x in range(num_tiles_x):
    for y in range(num_tiles_y):
        bbox_list.append(f"{starting_x + x * width * pix_size},{starting_y + y * height * pix_size},{starting_x + (x + 1) * width * pix_size},{starting_y + (y + 1) * height * pix_size}")

print(bbox_list)

num_tiles_x: 7, num_tiles_y: 5, total 35 tiles
['-13909093,5137296,-13759093,5287296', '-13909093,5287296,-13759093,5437296', '-13909093,5437296,-13759093,5587296', '-13909093,5587296,-13759093,5737296', '-13909093,5737296,-13759093,5887296', '-13759093,5137296,-13609093,5287296', '-13759093,5287296,-13609093,5437296', '-13759093,5437296,-13609093,5587296', '-13759093,5587296,-13609093,5737296', '-13759093,5737296,-13609093,5887296', '-13609093,5137296,-13459093,5287296', '-13609093,5287296,-13459093,5437296', '-13609093,5437296,-13459093,5587296', '-13609093,5587296,-13459093,5737296', '-13609093,5737296,-13459093,5887296', '-13459093,5137296,-13309093,5287296', '-13459093,5287296,-13309093,5437296', '-13459093,5437296,-13309093,5587296', '-13459093,5587296,-13309093,5737296', '-13459093,5737296,-13309093,5887296', '-13309093,5137296,-13159093,5287296', '-13309093,5287296,-13159093,5437296', '-13309093,5437296,-13159093,5587296', '-13309093,5587296,-13159093,5737296', '-13309093,57372

In [54]:
# base_url = "https://di-usfsdata.img.arcgis.com/arcgis/rest/services/CONUS_site_productivity_2018_masked_202106032103033/ImageServer/exportImage"
# base_url = "https://apps.fs.usda.gov/fsgisx03/rest/services/wo_nfs_gtac/Vegetation_Zones/ImageServer/exportImage"
# base_url = "https://apps.fs.usda.gov/fsgisx01/rest/services/RDW_LandscapeAndWildlife/Science_TCC_CONUS/ImageServer/exportImage"
base_url = "https://apps.fs.usda.gov/fsgisx01/rest/services/RDW_LandscapeAndWildlife/NLCD_TCC_CONUS/ImageServer/exportImage"

# tiles_path = "productivity_tiles/"
# tiles_path = "vegetation_tiles/"
tiles_path = "canopy_tiles/"

params = {
    'bbox': None,
    'bboxSR': 3857,
    'size': f"{width},{height}",
    'format': 'tiff',
    'pixelType': 'U8',
    'noData': 0,
    'noDataInterpretation': 'esriNoDataMatchAny',
    'interpolation': 'RSP_NearestNeighbor',
    'adjustAspectRatio': 'true',
    'validateExtent': 'false',
    'lercVersion': 1,
    'f': 'image'
}

def fetch_tile(bbox_list, n):
     # Update bbox parameter
    params['bbox'] = bbox_list[n]
    
    # Make request to fetch tile
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        # Save tile to file
        path_to_tile = f"{tiles_path}tile_{i}.tiff"
        with open(path_to_tile, "wb") as f:
            f.write(response.content)
        print(f"Saved tile {n}")
    else:
        print(f"Failed to fetch tile {n}: {response.status_code}")


In [56]:

# Create tiles directory if it doesn't exist
Path(tiles_path).mkdir(exist_ok=True)

for i, bbox in enumerate(bbox_list):
   fetch_tile(bbox_list, i)


Saved tile 0
Saved tile 1
Saved tile 2
Saved tile 3
Saved tile 4
Saved tile 5
Saved tile 6
Saved tile 7
Saved tile 8
Saved tile 9
Saved tile 10
Saved tile 11
Saved tile 12
Saved tile 13
Saved tile 14
Saved tile 15
Saved tile 16
Saved tile 17
Saved tile 18
Saved tile 19
Saved tile 20
Saved tile 21
Saved tile 22
Saved tile 23
Saved tile 24
Saved tile 25
Saved tile 26
Saved tile 27
Saved tile 28
Saved tile 29
Saved tile 30
Saved tile 31
Saved tile 32
Saved tile 33
Saved tile 34


Combine the tiles
```
gdalbuildvrt -srcnodata 0 -vrtnodata 0 tiles.vrt tiles/*.tiff
gdalwarp -s_srs EPSG:3857 -t_srs EPSG:4326 -srcnodata 0 -dstnodata 0 tiles.vrt tiles.tif
```

Tile: 

```
rio mbtiles --format WEBP --tile-size 512 --zoom-levels 1..13 -j 10 --progress-bar output_rgb.tif output.mbtiles 
```
